In [1]:
import sys
sys.path.append('./pytorch_sol2/')
#sys.path

In [2]:
from __future__ import print_function, division
import os, time, copy, sys
import pandas as pd
import numpy as np
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from skimage import io, transform
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import pickle
from pandas_ml import ConfusionMatrix
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split

from config import *
from GalaxiesDataset import *
from pytorchtools import EarlyStopping
from myGZ2_hubble import *
import warnings
warnings.filterwarnings('ignore')

In [3]:
from PIL import Image

remove = []
x = pd.read_csv('./t/gz2_hub.csv')
for i in x['asset_id']:
    if not os.path.isfile('./images_training_rev1/images/{}.jpg'.format(i)):
        remove.append(i)
x[~x['asset_id'].isin(remove)].to_csv('usethis.csv', index=False)    

In [4]:
transf = transforms.Compose([
    transforms.CenterCrop((224, 224)),
        transforms.Resize((105, 105)),
                             transforms.RandomHorizontalFlip(p=0.5),
                             transforms.RandomRotation(degrees=(0,360)),
                             transforms.RandomVerticalFlip(p=0.5),
                             transforms.ToTensor(),
                             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225])])

transformations = transforms.Compose([
    transforms.CenterCrop((224,224)),
    transforms.Resize((105, 105)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
    

In [5]:
#d = pd.read_csv('./t/training_solutions_rev1.csv')

In [6]:
train_ds = GalaxiesDataset(TRAIN_DIR, TRAIN_CSV, transform=transf)

In [7]:
train_ds.root_dir

'./datasets/images_training_rev1/images/'

In [8]:
cl=train_ds.classes_frame['hubble_type'].astype('category').cat.codes.tolist()
temp = pd.DataFrame(cl,columns=['cat'])
temp.head()

,cat
0,9
1,10
2,0
3,10
4,9


In [9]:
y = torch.from_numpy(temp.values)
train_idx, valid_idx= train_test_split(
np.arange(len(y)),
test_size=0.4,
shuffle=True,
    random_state=42,
stratify=y)

valid_idx, test_idx= train_test_split(
np.arange(len(y[valid_idx])),
test_size=0.5,
shuffle=True,
    random_state=42,
stratify=y[valid_idx])

In [10]:
train_idx.shape[0] + valid_idx.shape[0] +test_idx.shape[0]

239573

In [11]:
y.shape[0]

239573

In [12]:
size = len(train_ds)
indices = list(range(size))
split = int(np.floor(VALIDATION_SPLIT * size))
if SHUFFLE_DS:
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

test_split = int(np.floor(VALIDATION_SPLIT * len(train_indices)))
if SHUFFLE_DS:
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = train_indices[test_split:], train_indices[:test_split]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler   = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=0,
                                                 sampler=train_sampler)
val_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=0,
                                                 sampler=val_sampler)
unseen_dl = DataLoader(train_ds,batch_size=BATCH_SIZE, num_workers=0,
                                                 sampler=test_sampler)
print("Total: {} Train_dl: {} Validation_dl: {} Unseen dl: {} ".format(size, len(train_dl),                                                                    
                                                              len(val_dl),
                                                              len(unseen_dl)))

Total: 239573 Train_dl: 3594 Validation_dl: 1198 Unseen dl: 1198 


In [13]:
len(train_indices) + len(val_indices) + len(test_indices)

239573

In [14]:
#Change to 'val' if only want to evaluate, 'train' allows you to train the model
train_or_val = 'val'

In [15]:
if train_or_val == 'train':
    model = torch.load('./models/gz2_resnet50')
if train_or_val == 'val':
    model = torch.load('./models/gz2_hubble')
device = torch.device(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.00005)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=3,threshold=0.0001,factor=0.1, verbose=True)

In [16]:
gz_hub = GZ2_hubble(train_dl, val_dl, unseen_dl, model, optimizer, scheduler, criterion,device=device,BATCH_SIZE=BATCH_SIZE)
if train_or_val == 'train':
    gz_hub.train(n_epochs=300)

In [17]:
#For unweighted f1 score with percentage confusion matrix ...
#gz_hub.show_cf(f1_='notWeighted', cf_type='percentages')

#For weighted f1 score with numbers confusion matrix ...
gz_hub.show_cf(f1_ = 'weighted', cf_type='numbers')

Evaluation on unseen test set beginning:
Unseen accuracy: 68.17698006887196
Unseen + weighted f1 score: 0.6906027006809564
Confusion matrix:
[[7843  301    1    4    0    0   63   20    0  348  158]
 [ 759 7938  136    1    8    0  112   66    0  408  366]
 [  13  231 1010    0  348    0    6   12    0   83  211]
 [  46    9    0   16    0    0    1    0    0   45    2]
 [   2   36  264    0 2932    0   24   32    0   71  195]
 [   2   10    4    0    0    0    8    1    0    5    2]
 [ 228  293   33    2   40    0 2138  300    0  607  169]
 [ 141  281   38    1   75    0  870 2392    0  211 1072]
 [  22   15    0    0    3    0    0    2    0   42   13]
 [1089  924   70    0   62    0  301   67    0 2659  661]
 [ 601  903  121    2  119    0  126  542    0  788 5739]]
